# Virus Model
> Defines a model for modelling the outbreak of a virus

In [5]:
#| default_exp VirusModel

In [6]:
#| hide
from nbdev.showdoc import *

In [7]:
#| export
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import random

In [8]:
#| export
class VirusModel(Model):
    '''A model of virus spread through a population'''
    def __init__(self, width, height, initial_infected, recovery_rate):
        self.num_agents = width * height
        self.grid = MultiGrid(width, height, torus=True)
        self.schedule = RandomActivation(self)
        self.recovery_rate = recovery_rate

        # Create agents
        for i in range(self.num_agents):
            person = Person(i, self)
            self.schedule.add(person)
            # Place the person randomly on the grid
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            self.grid.place_agent(person, (x, y))
            # Infect some initial agents
            if i < initial_infected:
                person.infect()

        # Data collector for graphing results
        self.datacollector = DataCollector(
            {'Infected': lambda m: sum([1 for a in m.schedule.agents if a.infected]),
             'Immune': lambda m: sum([1 for a in m.schedule.agents if a.immune]),
             'Susceptible': lambda m: sum([1 for a in m.schedule.agents if not a.infected and not a.immune])}
        )

    def step(self):
        '''Advance the model by one step in time'''
        self.datacollector.collect(self)
        self.schedule.step()
        # Randomly recover some infected agents
        for agent in self.schedule.agents:
            if agent.infected and random.uniform(0, 1) < self.recovery_rate:
                agent.immune = True
                agent.infected = False

In [10]:
#| hide
import nbdev; nbdev.nbdev_export()